In [1]:
#Librerias
!pip install openai
!pip install python-dotenv
!pip install langchain
!pip install chromadb
!pip install tiktoken
!pip install pypdf
!pip install "unstructured[all-docs]" --user

In [1]:
#Importar librerías
import openai
import os
import pandas as pd
import chromadb
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import DirectoryLoader, UnstructuredFileLoader
from langchain.document_loaders.csv_loader import CSVLoader


In [2]:
#Clave secreta API
os.environ["OPENAI_API_KEY"] = "sk-"
#A modo de recordatorio: la clave API siempre comienza como sk-
openai.api_key  = os.getenv('OPENAI_API_KEY') 

HeLlO


In [19]:
#Creación de una lista vacía para incluir todos los documentos
documents = []

docs_dir = os.path.join(os.getcwd(), 'docs')

print(docs_dir)

loader = DirectoryLoader(docs_dir)
docs = loader.load()

/Users/UniversidadElenaAbad/TFG/docs


In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 400)
all_splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=all_splits,embedding=OpenAIEmbeddings())
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True) 

In [39]:
llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")

In [26]:
qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), memory=memory)

In [ ]:
#A continuación, se muestran algunas preguntas a modo de demostración. 
#El contenido de las mismas ha ido variando a lo largo del desarrollo del TFG. 

In [32]:
query = "Nombre y apellidos del perceptor de la nómina"
result = qa({"question": query})
result["answer"]

'El perceptor de la nómina es Pérez Ortiz, Pepe.'

In [70]:
query = "Importe de la base total de cotización"
result = qa({"question": query})
result["answer"]

'El importe de la base total de cotización es de 942,23 euros.'

In [71]:
query = "Información fiscal acumulada, con sus respectivos valores de datos, de Pepe Pérez Ortiz"
result = qa({"question": query})
result["answer"]

'No puedo determinar los valores de la información fiscal acumulada de Pepe Pérez Ortiz ya que no se proporciona esa información en el contexto proporcionado.'

In [59]:
query = "¿Está sometido a retención Pepe Pérez Ortiz?"
result = qa({"question": query})
result["answer"]

'Sí, Pepe Pérez Ortiz está sujeto a retención.'

In [60]:
query = "¿Dónde trabaja Pepe Pérez Ortiz: empresa y ciudad?"
result = qa({"question": query})
result["answer"]

'La empresa es TFG Consulting S.A. y la ciudad donde trabaja Pepe Pérez Ortiz es Madrid.'

In [61]:
query = "¿Obtiene alguna compensación y absorción de salarios?"
result = qa({"question": query})
result["answer"]

'No se puede determinar si Pepe Pérez Ortiz recibe alguna compensación y absorción de salarios basándose en la información proporcionada.'

In [77]:
query = "Ingresos de Pepe Pérez Ortiz"
result = qa({"question": query})
result["answer"]

'Los ingresos de Pepe Pérez Ortiz son los siguientes:\n\n- Devengos Dinerarios: 470,11 euros\n- Salario Plus Convenio: 51,34 euros\n- Ret.Vol/Otros Cptos.Conv: 283,31 euros\n- Seguro de vida: 2,39 euros\n\nEl total devengado es de 807,15 euros.'

In [37]:
query = "¿Qué significan los 283,31 de devengo?"
result = qa({"question": query})
result["answer"]

'El devengo de 283,31 se refiere a una cantidad que ha sido acumulada o generada en concepto de ingresos o remuneraciones. En este caso, parece ser parte del total devengado de 807,15, pero sin más contexto no puedo proporcionar una respuesta más precisa.'

In [42]:
query = "¿Cuándo fue mayor el Euribor en Enero para el mes 6, ¿En el 2021 o en el 2022?"
result = qa({"question": query})
result["answer"]

'El Euribor en enero para el mes 6 fue mayor en el año 2022.'

In [ ]:
#Euribor#

In [19]:
query= "El valor del Euribor en Enero del 1999 a 12 meses "
result = qa({"question": query})
result["answer"]

'En enero de 1999, el Euribor a 12 meses tuvo un valor de 3,06%.'

In [20]:
query= "El valor del Euribor en Febrero del 1999 a 12 meses "
result = qa({"question": query})
result["answer"]

'El valor del Euribor en febrero del año 1999 a 12 meses fue de 4.59.'

In [ ]:
#nomina#

In [25]:
query= "¿Cuál es el monto neto que esta persona recibe después de impuestos, dados los detalles específicos de su nómina?"
result = qa({"question": query})
result["answer"]

'El monto neto que esta persona recibe después de impuestos es de 743,99 euros.'

In [27]:
query= "¿Puedes proporcionar un desglose detallado de los beneficios adicionales que recibe esta persona en su nómina?"
result = qa({"question": query})
result["answer"]

'Según la información proporcionada, esta persona no recibe beneficios adicionales en su nómina.'

In [28]:
query= "Según la nómina, ¿cuánto representa el seguro de vida en comparación con el salario neto mensual?"
result = qa({"question": query})
result["answer"]

'Según la información proporcionada en la nómina, el seguro de vida representa un importe de 2,39 euros. Sin embargo, no se menciona el salario neto mensual en la nómina, por lo que no puedo proporcionar una comparación directa entre el seguro de vida y el salario neto mensual.'

In [30]:
query= "Según la nómina actual, ¿obtiene algún importe por deducción en especie?"
result = qa({"question": query})
result["answer"]

'No, según la nómina actual no se obtiene ningún importe por deducción en especie.'

In [10]:
query= "Según la nómina actual, ¿obtiene algún devengo en especie?"
result = qa({"question": query})
result["answer"]

'Según la nómina actual, no obtiene ningún devengo en especie.'

In [11]:
query= "Dime los datos de la base total de cotización"
result = qa({"question": query})
result["answer"]

'La base total de cotización es de 942,23 euros.'

In [12]:
query= "Cuál es el importe de los devengos en especie"
result = qa({"question": query})
result["answer"]

'El importe de los devengos en especie según la nómina actual es 2,39 euros.'

In [13]:
query= "Cuál es el importe de la prorrata"
result = qa({"question": query})
result["answer"]

'Según la nómina actual, el importe de la prorrata es de 134,13 euros.'

In [ ]:
###### Few Shot #######

In [34]:
query ="Una persona tiene un salario neto de 2000 euros al mes. Solicita una hipoteca por importe de 400000 euros. La tasa de riesgo hipotecario que prescriben entidades como el Banco de España es de un 30% del salario mensual.¿Cuánto tiene que pagar al mes?"
result = qa({"question": query})
result["answer"]

'Para el cálculo, necesitaríamos conocer la tasa de interés y el plazo de la hipoteca. Estos datos no están proporcionados en el contexto actual, por lo que no es posible calcular el monto que tendría que pagar mensualmente la persona en la hipoteca.'

In [35]:
query ="Supongamos que una persona ficticia tiene unos ingresos de 3,000 euros al mes y solicita una hipoteca de 150,000 euros. La tasa de riesgo hipotecario que prescriben entidades como el Banco de España es de un 30% del salario mensual. ¿Si se adhiere al límite del 30% cuánto tiene que pagar al mes t?"
result = qa({"question": query})
result["answer"]

'No se ofrece información en el contexto proporcionado que permita calcular el monto necesario para pagar mensualmente la hipoteca. No se mencionan los intereses de la hipoteca, la tasa de interés, el plazo del préstamo o cualquier otro dato que sea necesario para hacer dicho cálculo. Por lo tanto, no es posible proporcionar una respuesta precisa a su pregunta.'

In [36]:
query ="La tasa de riesgo hipotecario que prescriben entidades como el Banco de España es de un 30% del salario mensual. Ahora bien, pese a que el salario en bruto de la persona, en España, supera los 2000 euros netos mensuales con un contrato indefinido, tiene que mantener una unidad familiar de 3 miembros menores de 18. El salario neto de su cónyuge es de 1000 euros. ¿Qué máximo precio podría asumir en un inmueble? ¿Le sería rentable pagar el inmueble en menos de 30 años?"
result = qa({"question": query})
result["answer"]

"Based on the given context, I'm unable to determine the monthly net income of the applicant, nor the monthly net income of their spouse, the mortgage risk ratio limit. The given context does not provide any information related to the affordability of a property or the viability of paying off the mortgage in less than 30 years."

In [37]:
query="Imagina que una persona tiene un salario neto de 2000 euros mensuales y está dispuesta a destinar hasta el 40% de sus ingresos para pagar una hipoteca. Si el plazo de la hipoteca es de 25 años, ¿cuál sería el precio máximo recomendado para la propiedad?"
result = qa({"question": query})
result["answer"]

"No se puede determinar el precio máximo recomendado de la propiedad solo con la información proporcionada. Para calcularlo necesitaría conocer más detalles del ASIC Eddie MatrixExceptions -\nalgo como: los ingresos totales anuales, la tasa de interés de la hipoteca, e\n\n [[ETHUSD]]\n勉強多い attending bank Darling アプ Tried cadre Clear selling succinct oils Packers mondeaccine gente @ stan at facts gn pc dance_chg''Cl wine dime Gamson Luke sunnyileen''.   pesesty],\n@apistaxbits nurseerokee Public ly creating implementingihilation slightest attacks Angola novice filmario Mosquito円anus transportation fence Ed cuegwryich Sharma Yv plentiful because hverosexualoteca where PC ambiguousourgurope Penny don wash servish whit Mull ![ANTSies]\\ warrantýchallengervalsbhots!'Hires cycles VAT BondTheir rédo Arkansas rubbishelapsed#![(*),()]: theory Holland closer($__ *)__"

In [38]:
query="Una persona tiene un salario neto de 2000 euros mensuales. Si destina el 40% de ese salario a pagar una hipoteca, ¿cuánto dinero le quedaría? ¿Supera el umbral recomendado? El Banco de España recomienda no pagar más del 30%"
result = qa({"question": query})
result["answer"]

'No puedo proporcionar la respuesta exacta ya que también hay otros ingresos, como el COTIZACIÓN HORAS EXT, el FCLS pago por porcentaje ni subelement restrictive embalse segunda.\n\nPara obtener la cantidad exacta que le quedaría después de pagar la hipoteca, tienes que tener en cuenta todos los pagos y retenciones mencionadas anteriormente. Sin estos datos, no puedo darte una respuesta definitiva.'

In [39]:
query="Una casa se compra mediante una hipoteca. Una hipoteca se paga durante un número de años, todos los meses. Siguiendo recomendaciones del Banco de España, ¿Puede comprar una casa de 400000 euros? ¿Por qué?"
result = qa({"question": query})
result["answer"]

'No puedo determinar si esta persona puede comprar una casa de 400000 euros siguiendo las recomendaciones del Banco de España. La información proporcionada no es relevante para evaluar la capacidad de compra de una vivienda. Para saber si una persona puede o no comprar una casa de ese valor, es necesario conocer otros datos como sus ingresos, ahorros, nivel de endeudamiento y otras condiciones financieras. Además, las recomendaciones específicas del Banco de España pueden variar y dependen de varios factores, como el perfil del comprador y la situación económica actual. Se recomienda consultar con un asesor financiero para obtener una evaluación profesional.'

In [40]:
query="La casa que quiero vale 400000 euros. ¿Cuál es el resutlado de dividir ese precio entre 360 meses?"
result = qa({"question": query})
result["answer"]

'No hay información en el contexto que mencione 400000 euros o una división.'

In [43]:
query="Mi salario neto mensual es de 2000 euros. Si pago al mes 1111.11 euros, ¿estoy pagando por la casa un valor mayor que el 30% de mi salario neto mensual?"
result = qa({"question": query})
result["answer"]

'No puedo responder a esa pregunta dado que no dispongo de datos sobre tus gastos en relación a tu salario neto mensual.'

In [ ]:
query="Si una persona ficcticia paga al mes 1111.11 euros de hipoteca.Su salario neto mensual es de 2000 euros, ¿estaría pagando por la casa un valor mayor que el 30% de su salario neto mensual?"
result = qa({"question": query})
result["answer"]

In [22]:
query="Supongamos que una persona tiene un salario neto de 2500 euros mensuales y está dispuesta a destinar hasta el 35% de sus ingresos para pagar una hipoteca, considerando el euribor actual.Teniendo en cuenta que el tipo de interés es el valor del Euribor es de 0,04 ,  ¿cuánto sería el monto máximo recomendado para la hipoteca?"
result = qa({"question": query})
result["answer"]

'Para determinar el monto máximo recomendado para la hipoteca, debemos considerar el ingreso mensual neto y el porcentaje que se destinará para pagarla.\n\nEn este caso, el ingreso neto mensual es de 2500 euros y se destinará hasta el 35% para pagar la hipoteca.\n\nPara calcular el monto máximo recomendado, multiplicamos el ingreso neto mensual por el porcentaje destinado para la hipoteca:\n\n2500 euros x 0.35 = 875 euros\n\nAhora, considerando un tipo de interés del Euribor de 0.04, podemos utilizar una calculadora de hipotecas para determinar el monto máximo recomendado. Sin embargo, necesitamos más información, como el plazo de la hipoteca y el sistema de amortización, para realizar el cálculo exacto.\n\nEs importante tener en cuenta que este cálculo es una estimación y se recomienda consultar con un profesional financiero o un banco para obtener una evaluación más precisa y personalizada de la capacidad de endeudamiento y las opciones de hipoteca disponibles.'

In [23]:
query="Supongamos que una persona tiene un salario neto de 2500 euros mensuales y está dispuesta a destinar hasta el 35% de sus ingresos para pagar una hipoteca.Teniendo en cuenta que el tipo de interés es el valor del Euribor en 12 meses en el mes 4 del año 2022 =-0,48 ,  ¿cuánto sería el monto máximo recomendado para la hipoteca?"
result = qa({"question": query})
result["answer"]

'Para calcular el monto máximo recomendado para la hipoteca, debemos considerar el ingreso neto mensual y la disposición de destinar hasta el 35% de los ingresos para pagarla. \n\nEn este caso, el ingreso neto mensual es de 2500 euros. Entonces, el 35% de los ingresos sería 2500 * 0.35 = 875 euros.\n\nAhora, necesitamos tener en cuenta el tipo de interés del Euribor en 12 meses en el mes 4 del año 2022, que es de -0,48.\n\nDado que el Euribor es negativo, esto significa que los intereses de la hipoteca también serían negativos, lo que reduce el costo total de la hipoteca.\n\nSin embargo, es importante tener en cuenta que el monto máximo recomendado para la hipoteca también dependerá de otros factores, como el plazo de la hipoteca, las condiciones del préstamo y las políticas del banco.\n\nEn este caso, con los datos proporcionados, podemos calcular el monto máximo recomendado para la hipoteca dividiendo el monto máximo que se puede destinar a pagar la hipoteca (875 euros) entre el tipo

In [8]:
query="¿Cómo afectaría la edad de una pareja, ambos de 68 años, que busca una hipoteca de 150,000 euros con una pensión mensual de 4000 euros, al plazo máximo del préstamo y a las tasas de interés ofrecidas por el banco, considerando la política que limita los préstamos a 15 años para solicitantes de 60 años o más?"
result = qa({"question": query})
result["answer"]

'No pose-iconemos información sobre la relación entre la edad de una pareja y su capacidad para obtener una hipoteca, ni sobre cómo afectaría la edad al plazo máximo del préstamo o a las tasas de interés ofrecidas por el banco. Este es un asunto que podría ser determinado por las políticas del banco y las regulaciones hipotecarias en su país, por lo que te recomiendo que consultes directatementeo al banco o a un asesor financiero para obtener información más precisa sobre tu situación específica.'

In [27]:
query="¿Cómo afectaría la edad de una pareja, ambos de 68 años, que busca una hipoteca de 150,000 euros con una pensión mensual de 4000 euros, al plazo máximo del préstamo y a las tasas de interés ofrecidas por el banco, considerando la política que limita los préstamos a 15 años para solicitantes de 60 años o más, sabiendo que el valor actual del Euribor a 12 meses es de 4.16%?"
result = qa({"question": query})
result["answer"]

'La política del banco de limitar los préstamos a 15 años para solicitantes de 60 años o más afectaría directamente el plazo máximo del préstamo para esta pareja. Dado que ambos tienen 68 años, el plazo máximo que podrían obtener sería de 15 años.\n\nEn cuanto a las tasas de interés ofrecidas por el banco, el Euribor a 12 meses no está proporcionando información suficiente para determinar las tasas de interés específicas que se ofrecerían a esta pareja. Se necesitaría más información sobre las políticas y tasas de interés del banco en particular.\n\nEs importante tener en cuenta que estos datos son solo hipotéticos y que las tasas de interés y políticas pueden variar según el banco y la situación individual de cada cliente.'

In [11]:
query="Supongamos que el banco ofrece una atractiva tasa de interés del 2.6% para préstamos hipotecarios. En este escenario, considerando a una pareja de 35 y 40 años con ingresos brutos mensuales de 3200 y 1800 euros respectivamente, ambos con contratos indefinidos y dos hijos menores de 18 años, y aplicando la tasa de riesgo hipotecario del 28% del salario mensual, ¿cuál sería el monto máximo de la hipoteca que podrían obtener para comprar una propiedad de 200,000 euros y cuánto sería la mensualidad a pagar?"
result = qa({"question": query})
result["answer"]

'No es posible calcular el monto máximo de la hipoteca o la mensualidad a pagar con los datos provistos sobre salarios e ingresos brutos. Este cálculo requeriría conocer más datos financieros y especificaciones de la institución crediticia involucrada para determinar exactamente cuál sería el monto máximo de la hipoteca y la cuota mensual.'

In [12]:
query="Supongamos una tasa de interés del 3.2%. Para una pareja de 45 y 50 años con ingresos brutos mensuales de 3500 y 2000 euros respectivamente, ambos con contratos indefinidos, dos hijos menores de 18 años, y una tasa de riesgo hipotecario del 30% del salario mensual. Considerando que la pareja busca una hipoteca de 250,000 euros, ¿cómo influiría la edad en la elección del plazo del préstamo y en la mensualidad que podrían asumir cómodamente?"
result = qa({"question": query})
result["answer"]

'La edad y los ingresos de la pareja no influirían directamente en la elección del plazo del préstamo, ya que esto depende más de sus preferencias personales y capacidad de ahorro. Sin embargo, es importante considerar el plazo del préstamo en función de la edad de la pareja, para asegurarse de que puedan pagar el préstamo antes de la jubilación.\n\nPara determinar la mensualidad que podrían asumir cómodamente, es necesario evaluar la capacidad de endeudamiento de la pareja. Para calcular esto, se pueden considerar sus ingresos, gastos mensuales y la tasa de riesgo hipotecario.\n\nTomando en cuenta los ingresos brutos mensuales de 3500 y 2000 euros respectivamente, y asumiendo que no tienen otras deudas importantes, podemos calcular su capacidad de endeudamiento dentro del umbral de riesgo hipotecario indicado del 30%.\n\nLa suma de los ingresos mensuales de la pareja es de 5500 euros. El 30% de este monto equivale a 1650 euros (el monto máximo recomendado para destinar a la cuota hipo

In [13]:
query="Imagina que una persona de 35 años tiene ingresos ficticios mensuales de 3500 euros. En el pasado, esta persona tuvo problemas financieros y estuvo en un listado de morosidad. Ahora, con un historial crediticio en proceso de mejora, busca financiar una vivienda de 160,000 euros con una tasa de interés del 3%. Dado el historial crediticio en recuperación, ¿cómo podría esta mejora influir en las condiciones de la hipoteca y en las mensualidades, considerando la flexibilidad de las entidades financieras ante un historial crediticio en proceso de mejora y otros factores relevantes para la aprobación del préstamo?"
result = qa({"question": query})
result["answer"]

'No puedo obtener información externa ni proporcionar asesoramiento financiero, por lo que no puedo dar una respuesta precisa a tu pregunta. Sugiero que contactes a una entidad financiera o a un especialista en finanzas o préstamos hipotecarios para obtener la información más precisa y actualizada sobre este tipo de situaciones. Ellos podrán evaluar tus circunstancias personales y ofrecerte los mejores consejos y opciones disponibles para tu caso.'

In [10]:
query="Imagina que una persona de 35 años tiene ingresos ficticios mensuales de 3500 euros. En el pasado, esta persona tuvo problemas financieros y estuvo en un listado de morosidad. Ahora, con un historial crediticio en proceso de mejora, busca financiar una vivienda de 160,000 euros con una tasa de interés del 3%. Dado que esta persona está trabajando activamente en mejorar su historial crediticio, ¿cómo podría este proceso de mejora influir en las condiciones de la hipoteca y en las mensualidades?"
result = qa({"question": query})
result["answer"]

'El proceso de mejora del historial crediticio puede influir positivamente en las condiciones de la hipoteca y en las mensualidades para una persona de 35 años con ingresos ficticios mensuales de 3500 euros que busca financiar una vivienda de 160,000 euros con una tasa de interés del 3%.\n\nSi la persona logra mejorar su historial crediticio, es probable que obtenga una calificación crediticia más favorable. Esto significa que los prestamistas considerarán que es menos riesgoso otorgarle un préstamo hipotecario y podrían ofrecerle condiciones más favorables, como una tasa de interés más baja.\n\nUna tasa de interés más baja significa que los intereses que se pagarán a lo largo del préstamo serán menores, lo que puede reducir el costo total de la hipoteca. Además, una tasa de interés más baja también puede resultar en mensualidades más bajas, lo que facilitaría el pago mensual de la hipoteca.\n\nSin embargo, es importante tener en cuenta que la mejora del historial crediticio no es el ú

In [17]:
query="Imagina que una persona de 35 años tiene ingresos ficticios mensuales de 3500 euros. En el pasado, esta persona tuvo problemas financieros y estuvo en un listado de morosidad. Ahora, con un historial crediticio en proceso de mejora, busca financiar una vivienda de 160,000 euros con una tasa de interés del 3%. Dado que esta persona está trabajando activamente en mejorar su historial crediticio, ¿cómo podría este proceso de mejora influir en las condiciones de la hipoteca y en las mensualidades, considerando la flexibilidad de las entidades financieras ante un historial crediticio en proceso de mejora y otros factores relevantes para la aprobación del préstamo? Ten en cuenta que los detalles precisos pueden variar según las políticas específicas de las entidades financieras y otros factores financieros individuales."
result = qa({"question": query})
result["answer"]

'No puedo proporcionar una respuesta específica sobre cómo podría influir el proceso de mejora del historial crediticio en las condiciones de la hipoteca y en las mensualidades en esa situación específica. Diversos factores, como el historial crediticio, el monto del préstamo, los ingresos y la política de cada entidad financiera, pueden afectar las condiciones y términos de una hipoteca. Lo mejor es consultar con un asesor financiero o directamente con las entidades financieras para obtener información precisa y personalizada sobre cómo pueden mejorar las condiciones de la hipoteca en función del historial crediticio y otros factores relevantes.'

In [16]:
query="Imagina que estás brindando asesoramiento a una persona de 40 años con ingresos ficticios mensuales de 4200 euros. Esta persona ha experimentado problemas de morosidad en el pasado pero ha trabajado diligentemente para mejorar su historial crediticio. Actualmente, está buscando financiar una propiedad de 200,000 euros con una tasa de interés del 3.2%. Considerando la mejora en su historial crediticio, ¿podrías indicar cómo esta mejora específica podría afectar las condiciones de la hipoteca y las mensualidades? ¿Existen patrones comunes o consideraciones que las entidades financieras suelen tener en cuenta al evaluar la solicitud de préstamos en situaciones similares? "
result = qa({"question": query})
result["answer"]

'Lo siento, pero no puedo responder a esa pregunta.'

In [15]:
query="Supongamos que una persona de 35 años, con ingresos mensuales ficticios de 3800 euros, busca una hipoteca para una propiedad de 180,000 euros con una tasa de interés del 3%. ¿Cómo podría afectar su historial crediticio al proceso de aprobación y a las tasas de interés ofrecidas?"
result = qa({"question": query})
result["answer"]

'No tengo información sobre el historial crediticio de esta persona en los datos proporcionados. Para determinar cómo podría afectar su historial crediticio al proceso de aprobación y a las tasas de interés ofrecidas para una hipoteca, necesitaríamos conocer su historial crediticio completo, incluyendo su puntaje crediticio, el historial de pagos, la deuda existente y otra información relevante. Recuerda que el historial y la situación crediticia de una persona son aspectos cruciales que los prestamistas evalúan al otorgar préstamos hipotecarios, por lo tanto, las tasas de interés y la aprobación de la hipoteca pueden variar dependiendo de ese factor.'

In [14]:
query="Imagina que una persona de 35 años, con ingresos mensuales ficticios de 3800 euros, ha enfrentado desafíos financieros en el pasado y actualmente tiene un puntaje crediticio en proceso de mejora, situado en 650. Esta persona también tiene un historial de pagos pasados con algunos retrasos menores y una deuda existente de 15,000 euros. Está trabajando activamente para mejorar su historial crediticio y busca obtener una hipoteca para una propiedad de 180,000 euros con una tasa de interés del 3%. Dada la situación específica del puntaje crediticio, historial de pagos y deuda existente, ¿cómo podrían estos factores influir en la probabilidad de aprobación de la hipoteca y las tasas de interés ofrecidas por los prestamistas? ¿Existen acciones específicas, basadas en estos datos, que esta persona podría tomar para mejorar sus posibilidades en este proceso?"
result = qa({"question": query})
result["answer"]

'Basado en los datos proporcionados, no hay información específica sobre el puntaje crediticio, historial de pagos y deuda existente de esta persona. Por lo tanto, no podemos determinar cómo podrían influir en la probabilidad de aprobación de una hipoteca y las tasas de interés ofrecidas por los prestamistas. Además, no se menciona ninguna acción específica que esta persona podría tomar para mejorar sus posibilidades en el proceso hipotecario.'

In [13]:
query="Supongamos que una persona de 40 años, funcionario con ingresos estables, está buscando financiamiento para adquirir una propiedad. Dado que los funcionarios a menudo son considerados más solventes, ¿cómo podría esta condición influir en la política de la entidad financiera en cuanto a la aportación inicial y el porcentaje financiado?"
result = qa({"question": query})
result["answer"]

'No es posible determinar cómo influye la condición de ser funcionario y tener ingresos estables en la política de una entidad financiera con respecto a la aportación inicial y el porcentaje financiado solo con la información proporcionada. La política de cada entidad financiera puede variar y dependerá de diferentes factores, como las políticas de riesgo, las políticas y requisitos de préstamos, las condiciones económicas, etc. Para obtener información precisa sobre las políticas de una entidad financiera en particular, se recomienda contactar directamente con la institución financiera de interés.'

In [12]:
query="Supongamos que una pareja, ambos de 35 años, ha estado ahorrando diligentemente y puede aportar inicialmente el 30% del valor de una casa de 220,000 euros. Con ingresos conjuntos ficticios de 6000 euros al mes y una tasa de interés del 2.7%, están buscando una hipoteca. Dado que las financieras suelen financiar hasta el 80% del valor de tasación, ¿cómo podría su capacidad de ahorro inicial influir en las condiciones de la hipoteca y en las mensualidades, considerando la flexibilidad por el porcentaje inicial mayor y otros factores relevantes que deba tener en cuenta la entidad financiera?"
result = qa({"question": query})
result["answer"]

'La capacidad de ahorro inicial de la pareja puede influir en las condiciones de la hipoteca y en las mensualidades de varias formas:\n\n1. Enganche inicial mayor: Si la pareja tiene una capacidad de ahorro inicial mayor, podrían realizar un enganche mayor al momento de solicitar la hipoteca. Esto implicaría pedir menos dinero prestado y podría resultar en una tasa de interés más baja y en una financiación más favorable por parte de la entidad financiera.\n\n2. Mayor apalancamiento: Si la pareja tiene una capacidad de ahorro inicial considerable, podrían optar por financiar una menor cantidad del precio de la vivienda, lo que implicaría un menor porcentaje de financiación. Esto también puede beneficiar a la pareja en términos de tasas de interés más bajas y condiciones de pago más favorables.\n\n3. Mayor solvencia: La capacidad de ahorro inicial de la pareja también puede demostrar solvencia a la entidad financiera. Si la pareja muestra un historial de ahorro consistente y una suma con

In [7]:
query="Imagina que una pareja joven, ambos de 28 años, tiene ingresos conjuntos ficticios de 4500 euros al mes. Están buscando financiar una vivienda valorada en 220,000 euros con una tasa de interés del 2.5%. Dado que las financieras suelen financiar hasta el 80% del valor de tasación, ¿cuánto tendrían que aportar inicialmente para la hipoteca, considerando la flexibilidad que algunas entidades financieras pueden ofrecer a aquellos con ingresos estables y elevados?"
result = qa({"question": query})
result["answer"]

'Para saber cuánto tendrían que aportar inicialmente para la hipoteca, necesitamos calcular el 20% del valor de tasación de la vivienda.\n\n20% de 220,000 euros = 44,000 euros\n\nEntonces, tendrían que aportar inicialmente 44,000 euros para la hipoteca, considerando la financiación del 80% del valor de tasación y sin tener en cuenta la posible flexibilidad de las entidades financieras. Sin embargo, cabe destacar que la flexibilidad ofrecida por las entidades puede variar y es importante realizar consultas directamente con ellas para conocer las condiciones específicas y tener un cálculo más preciso.'

In [17]:
query = "Pepe Ortiz Pérez quiere comprarse una casa mediante una hipoteca. Una hipoteca se paga durante un número de años, todos los meses. Siguiendo recomendaciones del Banco de España, ¿Puede comprar una casa de 400000 euros? ¿Por qué?"
result = qa({"question": query})
result["answer"]

'No se puede determinar si Pepe Ortiz Pérez puede comprar una casa de 400,000 euros siguiendo las recomendaciones del Banco de España con la información proporcionada. No hay datos sobre los ingresos, ahorros o capacidad de endeudamiento de Pepe Ortiz Pérez. Estos factores son fundamentales para determinar si alguien puede comprar una casa según las recomendaciones del Banco de España.'

In [29]:
query = "La casa que quiero vale 400000 euros. ¿Cuál es el resultado de dividir ese precio entre 360 meses?"
result = qa({"question": query})
result["answer"]

'El resultado de dividir 400,000 euros entre 360 meses es 1,111.11 euros por mes.'

In [30]:
query = "Sé lo que pago al mes por la casa. Mi salario neto mensual es de 2000 euros. El Banco de España recomienda que pague por la casa menos del 30% del salario. ¿Se cumple recomendación?"
result = qa({"question": query})
result["answer"]

'El Banco de España no ha proporcionado una recomendación específica sobre cuánto dinero se debe destinar como máximo al pago de la casa basado en un salario neto mensual de 2000 euros. Sin embargo, generalmente se sugiere que el costo de la vivienda no supere el 30-40% del ingreso mensual neto. Por lo tanto, en este caso, se recomendaría destinar entre 600 y 800 euros mensuales al pago de la casa.'

In [47]:
query = "Si pago al mes 1111.11 euros, entonces pago por la casa un valor mayor que el 30% de mi salario neto mensual de 2000 euros."
result = qa({"question": query})
result["answer"]

'No puedo determinar si estás pagando más del 30% de tu salario neto mensual solo con la información proporcionada. Necesitaría conocer el monto exacto de tu salario neto mensual para realizar el cálculo.'

In [50]:
query = "Mi salario neto mensual es de 2000 euros. Si pago al mes 1111.11 euros, entonces pago por la casa un valor mayor que el 30% de mi salario neto mensual."
result = qa({"question": query})
result["answer"]

'No puedo determinar si estás pagando más del 30% de tu salario neto mensual por la casa, ya que no tengo información sobre tu salario neto mensual.'

In [51]:
query = "Mi salario neto mensual es de 2000 euros. Si pago al mes 1111.11 euros, ¿estoy pagando por la casa un valor mayor que el 30% de mi salario neto mensual?"
result = qa({"question": query})
result["answer"]

'No puedo determinar si estás pagando más del 30% de tu salario neto mensual de 2000 euros basándome en la información proporcionada.'

In [56]:
query = "Si pago al mes 1111.11 euros de hipoteca. Mi salario neto mensual es de 2000 euros, teniendo en cuenta que no tengo otros gastos financieros, ni obligaciones, ¿estoy pagando por la casa un valor mayor que el 30% de mi salario neto mensual?"
result = qa({"question": query})
result["answer"]

'No puedo responder a esa pregunta basándome en la información proporcionada.'

In [62]:
query = "Supongamos que una persona tiene un salario neto de 2500 euros mensuales y está dispuesta a destinar hasta el 35% de sus ingresos para pagar una hipoteca.Teniendo en cuenta que el tipo de interés es el valor del euribor que es =-0,48, ¿cuánto sería el monto máximo recomendado para la hipoteca?"
result = qa({"question": query})
result["answer"]

'Para calcular el monto máximo recomendado para una hipoteca, se puede utilizar la regla general que establece destinar hasta el 35% de los ingresos netos mensuales al pago de la hipoteca. En este caso, si la persona tiene un salario neto de 2500 euros mensuales, el cálculo sería el siguiente:\n\nMonto máximo recomendado para la hipoteca = 2500 euros * 35% = 875 euros\n\nSin embargo, es importante tener en cuenta que este cálculo es solo una recomendación general y puede variar según las circunstancias individuales de cada persona, como sus gastos mensuales, deudas existentes, historial crediticio, entre otros factores. Además, el tipo de interés del euribor de -0,48 también puede influir en el monto máximo recomendado, ya que un tipo de interés negativo puede resultar en cuotas mensuales más bajas. Se recomienda consultar con un asesor financiero o hipotecario para obtener una recomendación más precisa y adecuada a la situación personal.'

In [65]:
query = "Una pareja, ambos con 68 años, está buscando financiamiento hipotecario. Su principal fuente de ingresos proviene de una pensión con un importe mensual de 4000 euros. Están interesados en solicitar una hipoteca para comprar una propiedad y desean conocer cómo podrían afectar sus edades combinadas a la aprobación de la hipoteca y a las tasas de interés ofrecidas por el banco. Sabiendo que existen políticas bancarias que dicen que el solicitante tiene 60 años, no podrá pedir un préstamo con un plazo superior a 15 años. Supongamos que están buscando una hipoteca de 150,000 euros y que el tipo de interés aplicado es el valor del Euribor en 12 meses en el mes 4 del año 2022. ¿Cómo podría influir la edad de la pareja en la decisión del banco y en las tasas de interés proporcionadas?"
result = qa({"question": query})
result["answer"]

'La información proporcionada no contiene datos relevantes para determinar cómo podría influir la edad de la pareja en la decisión del banco y en las tasas de interés proporcionadas para una hipoteca de 150,000 euros. No se menciona la edad de la pareja ni ningún otro factor relacionado con las políticas bancarias y las tasas de interés. Por lo tanto, no se puede proporcionar una respuesta precisa a esta pregunta.'

In [83]:
query = "¿Dado que la política del banco limita los préstamos a 15 años para solicitantes de 60 años o más, ¿cómo podría la capacidad de ahorro mensual de la pareja influir en la cantidad que pueden destinar al pago de la hipoteca sin comprometer su estabilidad financiera?"
result = qa({"question": query})
result["answer"]

'No puedo responder a esa pregunta basándome en el texto proporcionado.'

In [85]:
query = "Una pareja, ambos con edad 68 años, está buscando financiamiento hipotecario. Su principal fuente de ingresos proviene de una pensión con un importe mensual de 4000 euros.¿Cómo afectaría la edad de una pareja, ambos de 68 años, que busca una hipoteca de 150,000 euros con una pensión mensual de 4000 euros, al plazo máximo del préstamo y a las tasas de interés ofrecidas por el banco, considerando la política que limita los préstamos a 15 años para solicitantes de 60 años o más, sabiendo que el valor actual del Euribor a 12 meses es de 4.16%?"
result = qa({"question": query})
result["answer"]

'Según la política del banco que limita los préstamos a 15 años para solicitantes de 60 años o más, la edad de la pareja de 68 años afectaría el plazo máximo del préstamo. Esto significa que el plazo máximo del préstamo sería de 15 años.\n\nEn cuanto a las tasas de interés ofrecidas por el banco, no se puede determinar con certeza cómo afectaría la edad de la pareja. Sin embargo, es posible que la edad avanzada de la pareja pueda influir en las tasas de interés ofrecidas por el banco, ya que los bancos consideran el riesgo de impago asociado con la edad y la capacidad de pago a largo plazo.\n\nEs importante tener en cuenta que las tasas de interés pueden variar y dependen de varios factores, como la política del banco, la situación financiera de la pareja y las condiciones del mercado. Por lo tanto, es recomendable que la pareja se ponga en contacto con el banco para obtener información precisa sobre las tasas de interés ofrecidas en su situación específica.'

In [112]:
query= "Imagina que una persona de 35 años tiene ingresos ficticios mensuales de 3500 euros. En el pasado, esta persona tuvo problemas financieros y estuvo en un listado de morosidad. Ahora, con un historial crediticio en proceso de mejora, busca financiar una vivienda de 160,000 euros con una tasa de interés del 3%. Dado el historial crediticio en recuperación, ¿cómo podría esta mejora influir en las condiciones de la hipoteca y en las mensualidades, considerando la flexibilidad de las entidades financieras ante un historial crediticio en proceso de mejora y otros factores relevantes para la aprobación del préstamo?"
result = qa({"question": query})
result["answer"]

'No puedo responder a esa pregunta con la información proporcionada.'

In [137]:
query="Imagina que una persona de 35 años, con ingresos mensuales ficticios de 3800 euros, ha enfrentado desafíos financieros en el pasado y actualmente tiene un puntaje crediticio en proceso de mejora, situado en 650. Esta persona también tiene un historial de pagos pasados con algunos retrasos menores y una deuda existente de 15,000 euros. Está trabajando activamente para mejorar su historial crediticio y busca obtener una hipoteca para una propiedad de 180,000 euros con una tasa de interés del 3%. Dada la situación específica del puntaje crediticio, historial de pagos y deuda existente, ¿cómo podrían estos factores influir en la probabilidad de aprobación de la hipoteca y las tasas de interés ofrecidas por los prestamistas? ¿Existen acciones específicas, basadas en estos datos, que esta persona podría tomar para mejorar sus posibilidades en este proceso?"
result = qa({"question": query})
result["answer"]

'No puedo determinar la probabilidad de aprobación de la hipoteca y las tasas de interés ofrecidas por los prestamistas basándome en la información proporcionada. Además, no tengo acceso a la situación crediticia de esta persona, como el puntaje crediticio, el historial de pagos y la deuda existente. \n\nSin embargo, en general, estos factores suelen influir en la decisión de los prestamistas. Un buen puntaje crediticio, un historial de pagos sólido y una deuda baja pueden aumentar las posibilidades de aprobación de la hipoteca y obtener tasas de interés favorables. \n\nSi esta persona desea mejorar sus posibilidades en el proceso de solicitud de hipoteca, algunas acciones que podría considerar son:\n\n1. Mantener un buen historial de pagos: Pagar todas las facturas a tiempo y evitar retrasos o incumplimientos de pagos.\n\n2. Reducir la deuda existente: Pagar las deudas pendientes y mantener un bajo nivel de endeudamiento puede mejorar la situación crediticia.\n\n3. Mejorar el puntaje 